In [15]:
import torch

print(torch.__version__)

1.0.0


In [85]:
data = torch.arange(24, dtype=torch.float32).view(2, 3, 4)

In [86]:
data

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [87]:
data[1]

tensor([[12., 13., 14., 15.],
        [16., 17., 18., 19.],
        [20., 21., 22., 23.]])

In [88]:
data

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]],

        [[12., 13., 14., 15.],
         [16., 17., 18., 19.],
         [20., 21., 22., 23.]]])

In [89]:
data.layout

torch.strided

In [90]:
data.ndimension()

3

In [91]:
data.dim()

3

In [92]:
data.size()

torch.Size([2, 3, 4])

In [93]:
# 以一维数组形式存储
data.storage()

 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
 9.0
 10.0
 11.0
 12.0
 13.0
 14.0
 15.0
 16.0
 17.0
 18.0
 19.0
 20.0
 21.0
 22.0
 23.0
[torch.FloatStorage of size 24]

In [94]:
data.size()

torch.Size([2, 3, 4])

In [95]:
data.stride()

(12, 4, 1)

In [96]:
data.dim()

3

In [97]:
data.stride(0)

12

In [98]:
data[1][2][2] 

tensor(22.)

##### dim=0

In [99]:
data[0]

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

In [100]:
data[1]

tensor([[12., 13., 14., 15.],
        [16., 17., 18., 19.],
        [20., 21., 22., 23.]])

##### dim=1

In [101]:
data[0][0]

tensor([0., 1., 2., 3.])

In [102]:
data[1][2]

tensor([20., 21., 22., 23.])

##### dim=2

In [103]:
data[0][0][3]

tensor(3.)

In [104]:
data.storage()

 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
 9.0
 10.0
 11.0
 12.0
 13.0
 14.0
 15.0
 16.0
 17.0
 18.0
 19.0
 20.0
 21.0
 22.0
 23.0
[torch.FloatStorage of size 24]

In [113]:
x = torch.Tensor().set_(data.storage(), storage_offset=3, size=torch.Size([2,3]), stride=(5,2))

In [114]:
x

tensor([[ 3.,  5.,  7.],
        [ 8., 10., 12.]])

In [117]:
x[1][2] 

tensor(12.)